<a href="https://colab.research.google.com/github/MarkAppprogrammer/random-ml/blob/main/transform_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus import brown
import torch
from gensim.test.utils import common_texts
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
from zeugma.embeddings import EmbeddingTransformer

#import torchtext

### Tokenization + Embedding
Tried to use openai's encoder but looking again at the paper all that is needed is to break it into words and embed using word2vec. Then postional encoding.

In [2]:
#Tokenization and Embedding
sample_input = "The dark brown fox jumped over the small road"

tokens = nltk.tokenize.word_tokenize(sample_input)

# cl100k_tokenizer = openai.cl100k_base()
# tokens = cl100k_tokenizer.encode(sample_input)

In [12]:
#nltk.download('brown')
nltk.download('word2vec_sample')
nltk.download('punkt')

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
#model = Word2Vec(sentences=common_texts, vector_size=512, window=5, min_count=1, workers=4)
# train_set = brown.sents()[:10000]
# model = Word2Vec(train_set, vector_size=512)

In [4]:
from nltk.data import find
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [13]:
def train_word2vec(sentences, vector_size=100, window=5, min_count=1, epochs=10):
    # Train Word2Vec model
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=1,
        workers=4,
        epochs=epochs
    )
    return model

In [21]:
def tokenize(data):
   return nltk.tokenize.word_tokenize(sample_input)

# note doesnt work with puncation
def embedder(tokens):
    model = train_word2vec([tokens], vector_size=512)

    vectors = [model.wv[token] for token in tokens if token in model.wv]

    return np.array(vectors)

In [22]:
#example
embedding = embedder(tokens)

### Postional Encoding
Note formulas in paper. For this example we use the sample sentence which was previously tokenized and we found mebeddings for.

In [25]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator) #forumulae from paper
    return P

#example
P = getPositionEncoding(9, 512, 10000)
embedding_with_pos = embedding + P

### Attenion Mechainsms

We need to make the scaled dot-product addtion and multi-head addditon used in the encoder and decoder stack of the transformer.

In [42]:
# def attention(Q, K, V):
#   return np.matmul(softmax((np.matmul(Q, K) * (1/np.sqrt(K.shape[1]))), axis=0), V)

def attention(Q, K, V):
    d_k = K.shape[-1]
    scores = np.matmul(Q, K.T) / np.sqrt(d_k)
    weights = softmax(scores, axis=-1)
    return np.matmul(weights, V)

In [50]:
def glorotInit(shape):
    fan_in, fan_out = shape[0], shape[1]
    limit = np.sqrt(6 / (fan_in + fan_out))
    return np.random.uniform(-limit, limit, size=shape)

def initWeights(d_model, d_k, d_v, h):
  # q, k, and v wegiths need to also have for all i shape should be (d_model, d_k, h) for W_Q
  W_Q = [glorotInit((d_model, d_k)) for _ in range(h)]
  W_K = [glorotInit((d_model, d_k)) for _ in range(h)]
  W_V = [glorotInit((d_model, d_v)) for _ in range(h)]
  W_O = glorotInit((h * d_v, d_model))

  return W_Q, W_K, W_V, W_O

def multiHeadAttention(Q, K, V, h, d_model):
  W_Q, W_K, W_V, W_O = initWeights(d_model, K.shape[1], V.shape[1], h)

  heads = []
  for i in range(h):
        Q_proj = np.matmul(Q, W_Q[i])
        K_proj = np.matmul(K, W_K[i])
        V_proj = np.matmul(V, W_V[i])

        heads.append(attention(Q_proj, K_proj, V_proj))

  concat = np.concatenate(heads, axis=-1)
  return np.matmul(concat, W_O)

### Encoder and Decoder Stacks

Now its time to combine everything we made above into the encoder and decoder stacks